In [20]:
import os
import numpy as np
from helpers import *
from fastai.vision.all import *

In [21]:
CUT_TO_SQUARE = True

RESIZE = False
SIZE = 256

LOCALIZE = False
LOCALIZE_BORDER = 0.20

REMAP = True
# BG RV MYO LV
ORIGINAL_CLASSES = [0, 1, 2, 3]
TARGET_CLASSES = [0, 1, 0, 0]

DATASET_NAME = 'rv'
DATASET_PATH = 'D:/Projects/PhD/datasets/ACDC/nii/training'
OUTPUT_PATH =  'D:/Projects/PhD/datasets/ACDC/png/training/' + DATASET_NAME

os.makedirs(OUTPUT_PATH, exist_ok=True)

In [22]:
reader = NiiReader()
images, image_names, configs = reader.read_folder(DATASET_PATH, '**/*[!4d][!gt].nii.gz')
masks, _, _ = reader.read_folder(DATASET_PATH, '**/*gt.nii.gz')

In [23]:
all_pixels = np.concatenate([img.flatten() for img in images])
p99 = np.percentile(all_pixels, 99)

In [24]:
counter = 0
for (image_array, mask_array, image_name, config) in zip(images, masks, image_names, configs):
    if CUT_TO_SQUARE:
        image_array = cut_to_square(image_array)
        mask_array = cut_to_square(mask_array)
    
    if REMAP:
        remap_mask(mask_array, ORIGINAL_CLASSES, TARGET_CLASSES)

    if np.all(mask_array == 0):
        print(f'The "{image_name}" image has no mask')
        continue

    if LOCALIZE:
        (min_row, max_row, min_col, max_col, width, height) = get_mask_bounding_box(mask_array, LOCALIZE_BORDER, CUT_TO_SQUARE)
        image_array = image_array[min_row:max_row, min_col:max_col]
        mask_array = mask_array[min_row:max_row, min_col:max_col]

    normalized_image = normalize_image(image_array, max_value=p99)

    image = PILImage.create(normalized_image.astype(np.uint8))
    mask = PILMask.create(mask_array.astype(np.uint8))
    
    if RESIZE:
        image = image.resize((SIZE, SIZE))
        mask = mask.resize((SIZE, SIZE))

    image_output = os.path.join(OUTPUT_PATH, 'images')
    mask_output = os.path.join(OUTPUT_PATH, 'masks')

    os.makedirs(image_output, exist_ok=True)
    os.makedirs(mask_output, exist_ok=True)

    image.save(os.path.join(image_output, image_name + '.png'))
    mask.save(os.path.join(mask_output, image_name + '.png'))

The "patient001_frame01_0" image has no mask


TypeError: normalize_image() got an unexpected keyword argument 'max_value'